In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x_train = mnist.train.images[:55000,:]
x_train.shape
#这里注意下是784个pixel training的时候shape要一致

(55000, 784)

In [4]:
len(mnist.train.images[0,:])

784

### DCGenerator

In [34]:
def generator(z,reuse = tf.AUTO_REUSE,is_training = True):
    depths = [1024, 512, 256, 128] + [784]
    with tf.variable_scope("Generator", reuse = reuse):
        with tf.variable_scope("g_fc1", reuse = reuse):
            output = tf.layers.dense(z, depths[0]*4*4,trainable = is_training)
            output = tf.reshape(output, [784, 4, 4, depths[0]])
            output = tf.nn.relu(tf.layers.batch_normalization(output,trainable = is_training))
        with tf.variable_scope("g_dc1", reuse = reuse):
            output = tf.layers.conv2d_transpose(output, depths[1], [5,5], strides =(2,2), padding ="SAME",
                                               trainable = is_training )
            output = tf.nn.relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("g_dc2", reuse = reuse):
            output = tf.layers.conv2d_transpose(output, depths[2], [5,5], strides = (2,2), padding ="SAME", 
                                               trainable = is_training )
            output = tf.nn.relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("g_dc3", reuse = reuse):
            output = tf.layers.conv2d_transpose(output,depths[3], [5,5], strides = (2,2), padding ="SAME",
                                               trainable = is_training )
            output = tf.nn.relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("g_dc4", reuse = reuse):
            output = tf.layers.conv2d_transpose(output,depths[4], [5,5], strides = (2,2), padding = "SAME", 
                                                trainable = is_training)
            g_logits = tf.nn.tanh(output)
            #output = tf.layers.dense(inputs = output, units = 784, activation = tf.nn.tanh, kernel_initializer =tf.contrib.layers.xavier_initializer(), use_bias = True)

    return   g_logits#output#g_logits

## Discriminator

In [64]:
def discriminator(x,reuse = tf.AUTO_REUSE,is_training = True):
    depths = [784] + [64, 128, 256, 512]
    with tf.variable_scope("Discriminator", reuse = reuse):
        with tf.variable_scope("d_cv1", reuse = reuse):
            output = tf.layers.conv2d(x, depths[1], [5,5], strides = (2,2), padding ="SAME",trainable = is_training)
            output = tf.nn.leaky_relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("d_cv2", reuse = reuse):
            output = tf.layers.conv2d(output, depths[2], [5,5], strides = (2,2), padding ="SAME", 
                                     trainable = is_training )
            output = tf.nn.leaky_relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("d_cv3", reuse = reuse):
            output = tf.layers.conv2d(output, depths[3], [5,5], strides = (2,2), padding = "SAME", 
                                     trainable = is_training )
            output = tf.nn.leaky_relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("d_cv4", reuse = reuse):
            output = tf.layers.conv2d(output, depths[4], [5,5], strides = (2,2), padding ="SAME", 
                                      trainable = is_training)
            output = tf.nn.leaky_relu(tf.layers.batch_normalization(output,trainable = is_training))

        with tf.variable_scope("d_fc1", reuse = reuse):
            output = tf.layers.flatten(output)
            d_logits = tf.layers.dense(output,1,trainable = is_training)
            #logit = tf.layers.dense(inputs = output, units = 1, kernel_initializer = tf.contrib.layers.xavier_initializer(), use_bias = True)

    return d_logits

### Loss Function

In [61]:
x = tf.placeholder(tf.float32,shape = [None,784])
z = tf.placeholder(tf.float32,shape = [None,100])

In [62]:
x = tf.placeholder(tf.float32 , shape = [None, 784], name ="Input_data")
x = tf.reshape(x, [-1] + [55000, 784,784])

In [63]:
x

<tf.Tensor 'Reshape_12:0' shape=(?, 55000, 784, 784) dtype=float32>

In [65]:
G_sample = generator(z) #取得生成器的生成结果
D_logit_real = discriminator(x) #取得判别器判别的真实手写数字的结果
D_logit_fake = discriminator(G_sample) #取得判别器判别的生成的手写数字的结果

ValueError: Trying to share variable Discriminator/d_cv1/conv2d/kernel, but specified shape (5, 5, 784, 64) and found shape (5, 5, 1, 64).

In [ ]:
depths = [1024, 512, 256, 128] + [x_train[2]]
output = tf.layers.dense(z, depths[0]*4*4)
output = tf.reshape(output, [784, 4, 4, depths[0]])
output = tf.nn.relu(tf.layers.batch_normalization(output))

In [ ]:
output.shape